In [ ]:
import ccxt
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import time
import os
from dotenv import load_dotenv

# Load environment variables for API keys
load_dotenv()

class CryptoTradingStrategy:
    def __init__(self, cmc_api_key, binance_api_key, binance_secret_key, timeframe='1d', lookback_days=30):
        """
        Initialize the trading strategy for Binance Futures testnet
        
        Parameters:
        -----------
        cmc_api_key : str
            CoinMarketCap API key
        binance_api_key : str
            Binance Futures testnet API key
        binance_secret_key : str
            Binance Futures testnet Secret key
        timeframe : str
            Timeframe for OHLCV data (default: '1d')
        lookback_days : int
            Number of days to look back for historical data (default: 30)
        """
        self.cmc_api_key = cmc_api_key
        
       # Initialize Binance Futures testnet
        self.exchange = ccxt.binance({
            'apiKey': binance_api_key,
            'secret': binance_secret_key,
            'enableRateLimit': True,
            'options': {
                'defaultType': 'future',
                'adjustForTimeDifference': True,
                'test': True  # Enable testnet
            },
        })
 
        # Explicitly set the testnet URLs
        self.exchange.urls['api'] = {
            'public': 'https://testnet.binancefuture.com/fapi/v1',
            'private': 'https://testnet.binancefuture.com/fapi/v1',
            'fapiPublic': 'https://testnet.binancefuture.com/fapi/v1',
            'fapiPrivate': 'https://testnet.binancefuture.com/fapi/v1'
        }
        self.exchange.urls['test'] = self.exchange.urls['api']
        
        self.timeframe = timeframe
        self.lookback_days = lookback_days
        self.top_cryptos_df = None
        self.historical_data = {}
        self.leverage = 1  # Default leverage
        self.position_size_pct = 0.05  # Default position size (5% of balance per position)
        self.open_positions = []
        
    def get_top_cryptos_by_marketcap(self, limit=50):
        """Fetch top cryptocurrencies by market cap from CoinMarketCap API"""
        url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
        parameters = {
            'start': '1',
            'limit': str(limit),
            'sort': 'market_cap',
            'sort_dir': 'desc',
            'convert': 'USD'
        }
        headers = {
            'Accepts': 'application/json',
            'X-CMC_PRO_API_KEY': self.cmc_api_key,
        }
        
        response = requests.get(url, headers=headers, params=parameters)
        data = response.json()
        
        if 'data' not in data:
            raise Exception(f"Error fetching data from CoinMarketCap: {data.get('status', {}).get('error_message', 'Unknown error')}")
        
        cryptos = []
        for crypto in data['data']:
            cryptos.append({
                'symbol': crypto['symbol'],
                'name': crypto['name'],
                'market_cap': crypto['quote']['USD']['market_cap'],
                'price': crypto['quote']['USD']['price'],
                'volume_24h': crypto['quote']['USD']['volume_24h'],
                'percent_change_24h': crypto['quote']['USD']['percent_change_24h']
            })
        
        self.top_cryptos_df = pd.DataFrame(cryptos)
        print(f"Retrieved top {len(cryptos)} cryptocurrencies by market cap")
        return self.top_cryptos_df
    
    def fetch_historical_data(self):
        """Fetch historical price data for top cryptocurrencies"""
        if self.top_cryptos_df is None:
            raise Exception("Please fetch top cryptocurrencies first using get_top_cryptos_by_marketcap()")
        
        # Calculate start timestamp (lookback days from now)
        since = int((datetime.now() - timedelta(days=self.lookback_days)).timestamp() * 1000)
        
        historical_data = {}
        available_symbols = []
        
        # Get all available futures markets on Binance
        markets = self.exchange.load_markets()
        futures_markets = {k: v for k, v in markets.items() if v['future']}
        
        for symbol in self.top_cryptos_df['symbol']:
            try:
                # Format symbol for Binance Futures (most use BTCUSDT format)
                market_symbol = f"{symbol}/USDT"
                perpetual_symbol = f"{symbol}USDT"
                
                if market_symbol in futures_markets:
                    print(f"Fetching historical data for {market_symbol}...")
                    ohlcv = self.exchange.fetch_ohlcv(market_symbol, self.timeframe, since)
                    
                    # Convert to DataFrame
                    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                    df.set_index('timestamp', inplace=True)
                    
                    historical_data[symbol] = df
                    available_symbols.append({
                        'symbol': symbol, 
                        'market': market_symbol,
                        'perpetual': perpetual_symbol
                    })
                    
                    # Respect rate limits
                    time.sleep(self.exchange.rateLimit / 1000)
                else:
                    print(f"Market {market_symbol} not available on Binance Futures")
            except Exception as e:
                print(f"Error fetching data for {symbol}: {str(e)}")
        
        self.historical_data = historical_data
        self.available_symbols = pd.DataFrame(available_symbols)
        
        # Create a combined DataFrame of close prices
        close_prices = {}
        for symbol, df in historical_data.items():
            close_prices[symbol] = df['close']
        
        combined_df = pd.DataFrame(close_prices)
        
        return combined_df
    
    def identify_trading_signals(self, days=20):
        """
        Identify cryptocurrencies at N-day highs (buy signals) and N-day lows (sell signals)
        
        Parameters:
        -----------
        days : int
            Number of days to calculate highs and lows (default: 20)
        
        Returns:
        --------
        tuple
            (buy_signals, sell_signals) as DataFrames with symbol information
        """
        if not self.historical_data:
            raise Exception("Please fetch historical data first using fetch_historical_data()")
        
        buy_signals = []
        sell_signals = []
        
        for symbol, df in self.historical_data.items():
            if len(df) < days:
                print(f"Not enough data for {symbol}, skipping (has {len(df)} days, need {days})")
                continue
                
            # Calculate if current price is at N-day high or low
            current_price = df['close'].iloc[-1]
            period_high = df['close'].rolling(window=days).max().iloc[-1]
            period_low = df['close'].rolling(window=days).min().iloc[-1]
            
            # Get market symbol
            market_info = self.available_symbols[self.available_symbols['symbol'] == symbol]
            if len(market_info) == 0:
                continue
                
            market_symbol = market_info['market'].values[0]
            perpetual_symbol = market_info['perpetual'].values[0]
            
            # Check for signals
            signal_data = {
                'symbol': symbol,
                'market': market_symbol,
                'perpetual': perpetual_symbol,
                'current_price': current_price,
                'period_high': period_high,
                'period_low': period_low
            }
            
            if current_price >= period_high:
                buy_signals.append(signal_data)
            elif current_price <= period_low:
                sell_signals.append(signal_data)
        
        return pd.DataFrame(buy_signals) if buy_signals else pd.DataFrame(), pd.DataFrame(sell_signals) if sell_signals else pd.DataFrame()
    
    def set_leverage(self, leverage, symbol=None):
        """
        Set leverage for trading
    
        Parameters:
         -----------
        leverage : int
        Leverage value (1-125 depending on symbol)
        symbol : str, optional
        Trading pair symbol (e.g., 'BTCUSDT'). If None, sets for all open positions.
        """
        try:
            if symbol:
                # Format symbol properly (remove the / if present)
                formatted_symbol = symbol.replace('/', '')
                response = self.exchange.fapiPrivatePostLeverage({
                    'symbol': formatted_symbol,
                    'leverage': leverage
                })
                print(f"Leverage for {symbol} set to {leverage}x")
            else:
                # If no symbol provided, set leverage for all available symbols
                positions = self.get_open_positions()
                if not positions.empty:
                    for _, pos in positions.iterrows():
                        symbol = pos['symbol']
                        formatted_symbol = symbol.replace('/', '')
                        response = self.exchange.fapiPrivatePostLeverage({
                            'symbol': formatted_symbol,
                            'leverage': leverage
                        })
                        print(f"Leverage for all positions set to {leverage}x")
                    else:
                        print("No open positions to set leverage for")
        
            self.leverage = leverage
        except Exception as e:
            print(f"Error setting leverage: {str(e)}")

    
    def set_position_size(self, percentage):
        """Set position size as percentage of account balance"""
        if percentage <= 0 or percentage > 100:
            raise ValueError("Position size percentage must be between 0 and 100")
        self.position_size_pct = percentage / 100
        print(f"Position size set to {percentage}% of account balance")
    
    def get_account_balance(self):
        """Get account balance from Binance Futures testnet"""
        try:
            balance = self.exchange.fetch_balance()
            # For futures, get USDT balance
            usdt_balance = next((item for item in balance['info']['assets'] if item['asset'] == 'USDT'), None)
            if usdt_balance:
                return float(usdt_balance['availableBalance'])
            return float(balance['USDT']['free']) if 'USDT' in balance else 0
        except Exception as e:
            print(f"Error getting account balance: {str(e)}")
            return 0
    
    def get_open_positions(self):
        """Get currently open positions"""
        try:
            positions = self.exchange.fetch_positions()
            # Filter for positions with non-zero size
            open_positions = [p for p in positions if float(p['info']['positionAmt']) != 0]
            
            # Format for display
            formatted_positions = []
            for pos in open_positions:
                symbol = pos['symbol']
                side = 'LONG' if float(pos['info']['positionAmt']) > 0 else 'SHORT'
                entry_price = float(pos['entryPrice'])
                amount = abs(float(pos['info']['positionAmt']))
                pnl = float(pos['info']['unrealizedProfit'])
                roe = float(pos['info']['roe']) * 100 if 'roe' in pos['info'] else None
                
                formatted_positions.append({
                    'symbol': symbol,
                    'side': side,
                    'entry_price': entry_price,
                    'amount': amount,
                    'pnl': pnl,
                    'roe': roe
                })
            
            self.open_positions = formatted_positions
            return pd.DataFrame(formatted_positions) if formatted_positions else pd.DataFrame()
        except Exception as e:
            print(f"Error getting open positions: {str(e)}")
            return pd.DataFrame()
    
    def place_order(self, symbol, side, amount):
        """
        Place an order on Binance Futures testnet
    
        Parameters:
        -----------
        symbol : str
            Trading pair symbol (e.g., 'BTC/USDT')
        side : str
            'buy' or 'sell'
        amount : float
            Amount to trade
    
        Returns:
        --------
        dict
            Order response
        """
        try:
            # Set leverage for the symbol
            formatted_symbol = symbol.replace('/', '')
            self.exchange.fapiPrivatePostLeverage({
                'symbol': formatted_symbol,
                'leverage': self.leverage
            })
        
            # Place the order
            order = self.exchange.create_order(
                symbol=symbol,
                type='MARKET',
                side=side,
                amount=amount
            )
        
            print(f"Placed {side.upper()} order for {amount} {symbol} successfully")
            return order
        except Exception as e:
            print(f"Error placing order: {str(e)}")
            return None
    
    def close_position(self, symbol):
        """
        Close an open position
        
        Parameters:
        -----------
        symbol : str
            Trading pair symbol (e.g., 'BTC/USDT')
        
        Returns:
        --------
        dict
            Order response
        """
        try:
            # Get position information
            positions = self.exchange.fetch_positions([symbol])
            if not positions or float(positions[0]['info']['positionAmt']) == 0:
                print(f"No open position for {symbol}")
                return None
            
            # Determine side and amount for closing
            position_amount = float(positions[0]['info']['positionAmt'])
            close_side = 'sell' if position_amount > 0 else 'buy'
            close_amount = abs(position_amount)
            
            # Place order to close position
            order = self.exchange.create_order(
                symbol=symbol,
                type='MARKET',
                side=close_side,
                amount=close_amount,
                params={'reduceOnly': True}
            )
            
            print(f"Closed position for {symbol} successfully")
            return order
        except Exception as e:
            print(f"Error closing position: {str(e)}")
            return None
    
    def execute_strategy(self, days=20):
        """
        Execute the trading strategy based on identified signals
        
        Parameters:
        -----------
        days : int
            Number of days to calculate highs and lows (default: 20)
            
        Returns:
        --------
        tuple
            (executed_buys, executed_sells) as lists of order responses
        """
        # Get account balance
        balance = self.get_account_balance()
        print(f"Account balance: {balance} USDT")
        
        if balance <= 0:
            print("Insufficient account balance")
            return [], []
        
        # Identify signals
        buy_signals, sell_signals = self.identify_trading_signals(days=days)
        
        # Get current positions
        current_positions = self.get_open_positions()
        current_symbols = set(current_positions['symbol'].tolist()) if not current_positions.empty else set()
        
        executed_buys = []
        executed_sells = []
        
        # Execute sell signals (close long positions at 20-day lows)
        if not sell_signals.empty:
            for _, signal in sell_signals.iterrows():
                symbol = signal['market']
                if symbol in current_symbols:
                    print(f"Closing long position for {symbol} (20-day low)")
                    order = self.close_position(symbol)
                    if order:
                        executed_sells.append(order)
        
        # Execute buy signals (open long positions at 20-day highs)
        if not buy_signals.empty:
            for _, signal in buy_signals.iterrows():
                symbol = signal['market']
                current_price = signal['current_price']
                
                # Skip if already in position
                if symbol in current_symbols:
                    print(f"Already in position for {symbol}, skipping buy signal")
                    continue
                
                # Calculate position size
                position_size_usd = balance * self.position_size_pct
                amount = position_size_usd / current_price
                
                # Round to appropriate precision
                try:
                    market_info = self.exchange.market(symbol)
                    amount_precision = market_info['precision']['amount']
                    amount = round(amount, amount_precision)
                except:
                    # Default precision if market info not available
                    amount = round(amount, 4)
                
                # Place buy order
                print(f"Opening long position for {symbol} at {current_price} (20-day high)")
                order = self.place_order(symbol, 'buy', amount)
                if order:
                    executed_buys.append(order)
        
        return executed_buys, executed_sells
    
    def run_interactive(self):
        """Run interactive trading session"""
        print("==== Interactive Crypto Trading Strategy ====")
        print("Using Binance Futures Testnet")
        
        while True:
            print("\nMenu:")
            print("1. Get top cryptocurrencies by market cap")
            print("2. Fetch historical data")
            print("3. Show trading signals")
            print("4. Show account balance")
            print("5. Show open positions")
            print("6. Set leverage")
            print("7. Set position size")
            print("8. Execute strategy")
            print("9. Close specific position")
            print("10. Close all positions")
            print("0. Exit")
            
            choice = input("\nEnter your choice: ")
            
            try:
                if choice == '1':
                    limit = int(input("Enter number of top cryptocurrencies to fetch (default 50): ") or 50)
                    self.get_top_cryptos_by_marketcap(limit=limit)
                    if not self.top_cryptos_df.empty:
                        print("\nTop 10 cryptocurrencies by market cap:")
                        print(self.top_cryptos_df.head(10)[['symbol', 'name', 'market_cap', 'price']])
                
                elif choice == '2':
                    self.fetch_historical_data()
                    if self.historical_data:
                        print(f"\nCollected historical data for {len(self.historical_data)} cryptocurrencies")
                        print("\nAvailable symbols for trading:")
                        print(self.available_symbols)
                
                elif choice == '3':
                    days = int(input("Enter number of days for high/low calculation (default 20): ") or 20)
                    buy_signals, sell_signals = self.identify_trading_signals(days=days)
                    
                    print("\nBuy signals (cryptocurrencies at 20-day highs):")
                    if not buy_signals.empty:
                        print(buy_signals[['symbol', 'market', 'current_price', 'period_high']])
                    else:
                        print("No buy signals")
                        
                    print("\nSell signals (cryptocurrencies at 20-day lows):")
                    if not sell_signals.empty:
                        print(sell_signals[['symbol', 'market', 'current_price', 'period_low']])
                    else:
                        print("No sell signals")
                
                elif choice == '4':
                    balance = self.get_account_balance()
                    print(f"\nAccount balance: {balance} USDT")
                
                elif choice == '5':
                    positions = self.get_open_positions()
                    print("\nOpen positions:")
                    if not positions.empty:
                        print(positions)
                    else:
                        print("No open positions")
                
                elif choice == '6':
                    leverage = int(input("Enter leverage (1-20): "))
                    self.set_leverage(leverage)
                
                elif choice == '7':
                    pct = float(input("Enter position size as percentage of account balance (0-100): "))
                    self.set_position_size(pct)
                
                elif choice == '8':
                    days = int(input("Enter number of days for high/low calculation (default 20): ") or 20)
                    executed_buys, executed_sells = self.execute_strategy(days=days)
                    
                    print(f"\nExecuted {len(executed_buys)} buy orders and {len(executed_sells)} sell orders")
                
                elif choice == '9':
                    # Show available positions
                    positions = self.get_open_positions()
                    if positions.empty:
                        print("No open positions to close")
                        continue
                        
                    print("\nOpen positions:")
                    print(positions)
                    
                    symbol = input("Enter symbol to close (e.g., BTC/USDT): ")
                    self.close_position(symbol)
                
                elif choice == '10':
                    positions = self.get_open_positions()
                    if positions.empty:
                        print("No open positions to close")
                        continue
                        
                    for _, pos in positions.iterrows():
                        symbol = pos['symbol']
                        print(f"Closing position for {symbol}")
                        self.close_position(symbol)
                
                elif choice == '0':
                    print("Exiting...")
                    break
                
                else:
                    print("Invalid choice, please try again")
                    
            except Exception as e:
                print(f"Error: {str(e)}")
            
            # Pause before showing menu again
            input("\nPress Enter to continue...")

# Example usage
if __name__ == "__main__":
    # Get API keys from environment variables
    cmc_api_key = os.getenv('CMC_API_KEY')
    binance_api_key = os.getenv('BINANCE_FUTURES_TESTNET_API_KEY')
    binance_secret_key = os.getenv('BINANCE_FUTURES_TESTNET_SECRET_KEY')
    
    # Prompt for API keys if not found in environment
    if not cmc_api_key:
        cmc_api_key = input("Enter your CoinMarketCap API key: ")
    if not binance_api_key:
        binance_api_key = input("Enter your Binance Futures testnet API key: ")
    if not binance_secret_key:
        binance_secret_key = input("Enter your Binance Futures testnet secret key: ")
    
    # Initialize strategy
    strategy = CryptoTradingStrategy(
        cmc_api_key=cmc_api_key,
        binance_api_key=binance_api_key,
        binance_secret_key=binance_secret_key
    )
    
    # Run interactive session
    strategy.run_interactive()

Enter your CoinMarketCap API key:  exit


In [4]:
import ccxt
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import time
import os
from dotenv import load_dotenv

# Load environment variables for API keys
load_dotenv()

class CryptoTradingStrategy:
    def __init__(self, cmc_api_key, binance_api_key, binance_secret_key, timeframe='1d', lookback_days=30):
        """
        Initialize the trading strategy for Binance Futures testnet
        
        Parameters:
        -----------
        cmc_api_key : str
            CoinMarketCap API key
        binance_api_key : str
            Binance Futures testnet API key
        binance_secret_key : str
            Binance Futures testnet Secret key
        timeframe : str
            Timeframe for OHLCV data (default: '1d')
        lookback_days : int
            Number of days to look back for historical data (default: 30)
        """
        self.cmc_api_key = cmc_api_key
        
        # Initialize Binance Futures testnet using the proper class
        # Use BinanceFutures (usdm) instead of regular Binance
        self.exchange = ccxt.binanceusdm({
            'apiKey': binance_api_key,
            'secret': binance_secret_key,
            'enableRateLimit': True,
            'options': {
                'adjustForTimeDifference': True,
                'recvWindow': 10000,  # Increase receive window to avoid timestamp errors
            }
        })
        
        # Enable testnet mode - this uses the correct testnet URLs
        self.exchange.set_sandbox_mode(True)
        
        self.timeframe = timeframe
        self.lookback_days = lookback_days
        self.top_cryptos_df = None
        self.historical_data = {}
        self.leverage = 1  # Default leverage
        self.position_size_pct = 0.05  # Default position size (5% of balance per position)
        self.open_positions = []
        
    def get_top_cryptos_by_marketcap(self, limit=50):
        """Fetch top cryptocurrencies by market cap from CoinMarketCap API"""
        url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
        parameters = {
            'start': '1',
            'limit': str(limit),
            'sort': 'market_cap',
            'sort_dir': 'desc',
            'convert': 'USD'
        }
        headers = {
            'Accepts': 'application/json',
            'X-CMC_PRO_API_KEY': self.cmc_api_key,
        }
        
        response = requests.get(url, headers=headers, params=parameters)
        data = response.json()
        
        if 'data' not in data:
            raise Exception(f"Error fetching data from CoinMarketCap: {data.get('status', {}).get('error_message', 'Unknown error')}")
        
        cryptos = []
        for crypto in data['data']:
            cryptos.append({
                'symbol': crypto['symbol'],
                'name': crypto['name'],
                'market_cap': crypto['quote']['USD']['market_cap'],
                'price': crypto['quote']['USD']['price'],
                'volume_24h': crypto['quote']['USD']['volume_24h'],
                'percent_change_24h': crypto['quote']['USD']['percent_change_24h']
            })
        
        self.top_cryptos_df = pd.DataFrame(cryptos)
        print(f"Retrieved top {len(cryptos)} cryptocurrencies by market cap")
        return self.top_cryptos_df
    
    def fetch_historical_data(self):
        """Fetch historical price data for top cryptocurrencies"""
        if self.top_cryptos_df is None:
            raise Exception("Please fetch top cryptocurrencies first using get_top_cryptos_by_marketcap()")
    
        # Calculate start timestamp (lookback days from now)
        since = int((datetime.now() - timedelta(days=self.lookback_days)).timestamp() * 1000)
    
        historical_data = {}
        available_symbols = []
    
        # Get all available futures markets on Binance
        markets = self.exchange.load_markets()
    
        # Print some of the available markets to debug
        print("Available market examples:", list(markets.keys())[:5])
    
        for symbol in self.top_cryptos_df['symbol']:
            try:
                # For Binance Futures, use the correct format: BTC/USDT:USDT
                futures_symbol = f"{symbol}/USDT:USDT"
                perpetual_symbol = f"{symbol}USDT"
            
                # Try alternative format if needed
                if futures_symbol not in markets:
                    futures_symbol = f"{symbol}USDT"  # Try raw format like BTCUSDT
                    if futures_symbol in markets:
                        futures_symbol = markets[futures_symbol]['symbol']  # Get normalized symbol
                    else:
                        # Try other common formats
                        alt_symbols = [
                            f"{symbol}/USDT",
                            f"{symbol}USDT",
                            f"{symbol}/USDT:USD",
                        ]
                    
                        for alt_symbol in alt_symbols:
                            if alt_symbol in markets:
                                futures_symbol = alt_symbol
                                break
                
                if futures_symbol in markets:
                    print(f"Fetching historical data for {futures_symbol}...")
                    ohlcv = self.exchange.fetch_ohlcv(futures_symbol, self.timeframe, since)
                
                    # Convert to DataFrame
                    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                    df.set_index('timestamp', inplace=True)
                
                    historical_data[symbol] = df
                    available_symbols.append({
                        'symbol': symbol, 
                        'market': futures_symbol,
                        'perpetual': perpetual_symbol
                    })
                
                    # Respect rate limits
                    time.sleep(self.exchange.rateLimit / 1000)
                else:
                    print(f"Market for {symbol} not available on Binance Futures. Tried {futures_symbol}")
            except Exception as e:
                print(f"Error fetching data for {symbol}: {str(e)}")
    
        self.historical_data = historical_data
        self.available_symbols = pd.DataFrame(available_symbols)
    
        # Create a combined DataFrame of close prices
        close_prices = {}
        for symbol, df in historical_data.items():
            close_prices[symbol] = df['close']
    
        combined_df = pd.DataFrame(close_prices)
    
        return combined_df

    def identify_trading_signals(self, days=20):
        """
        Identify cryptocurrencies at N-day highs (buy signals) and N-day lows (sell signals)
        
        Parameters:
        -----------
        days : int
            Number of days to calculate highs and lows (default: 20)
        
        Returns:
        --------
        tuple
            (buy_signals, sell_signals) as DataFrames with symbol information
        """
        if not self.historical_data:
            raise Exception("Please fetch historical data first using fetch_historical_data()")
        
        buy_signals = []
        sell_signals = []
        
        for symbol, df in self.historical_data.items():
            if len(df) < days:
                print(f"Not enough data for {symbol}, skipping (has {len(df)} days, need {days})")
                continue
                
            # Calculate if current price is at N-day high or low
            current_price = df['close'].iloc[-1]
            period_high = df['close'].rolling(window=days).max().iloc[-1]
            period_low = df['close'].rolling(window=days).min().iloc[-1]
            
            # Get market symbol
            market_info = self.available_symbols[self.available_symbols['symbol'] == symbol]
            if len(market_info) == 0:
                continue
                
            market_symbol = market_info['market'].values[0]
            perpetual_symbol = market_info['perpetual'].values[0]
            
            # Check for signals
            signal_data = {
                'symbol': symbol,
                'market': market_symbol,
                'perpetual': perpetual_symbol,
                'current_price': current_price,
                'period_high': period_high,
                'period_low': period_low
            }
            
            if current_price >= period_high:
                buy_signals.append(signal_data)
            elif current_price <= period_low:
                sell_signals.append(signal_data)
        
        return pd.DataFrame(buy_signals) if buy_signals else pd.DataFrame(), pd.DataFrame(sell_signals) if sell_signals else pd.DataFrame()
    
    def set_leverage(self, leverage):
        """Set leverage for trading"""
        self.leverage = leverage
        print(f"Leverage set to {leverage}x")
        # Note: Actual leverage setting happens per symbol when placing orders
    
    def set_position_size(self, percentage):
        """Set position size as percentage of account balance"""
        if percentage <= 0 or percentage > 100:
            raise ValueError("Position size percentage must be between 0 and 100")
        self.position_size_pct = percentage / 100
        print(f"Position size set to {percentage}% of account balance")
    
    def get_account_balance(self):
        """Get account balance from Binance Futures testnet"""
        try:
            balance = self.exchange.fetch_balance()
            # For futures, get USDT balance
            if 'USDT' in balance['total']:
                return float(balance['free']['USDT'])
            return 0
        except Exception as e:
            print(f"Error getting account balance: {str(e)}")
            return 0
    
    def get_open_positions(self):
        """Get currently open positions"""
        try:
            positions = self.exchange.fetch_positions()
            # Filter for positions with non-zero size
            open_positions = [p for p in positions if abs(float(p['contracts'])) > 0]
            
            # Format for display
            formatted_positions = []
            for pos in open_positions:
                symbol = pos['symbol']
                side = pos['side'].upper()
                entry_price = float(pos['entryPrice'])
                amount = abs(float(pos['contracts']))
                pnl = float(pos['unrealizedPnl'])
                roe = float(pos['percentage']) if 'percentage' in pos else None
                
                formatted_positions.append({
                    'symbol': symbol,
                    'side': side,
                    'entry_price': entry_price,
                    'amount': amount,
                    'pnl': pnl,
                    'roe': roe
                })
            
            self.open_positions = formatted_positions
            return pd.DataFrame(formatted_positions) if formatted_positions else pd.DataFrame()
        except Exception as e:
            print(f"Error getting open positions: {str(e)}")
            return pd.DataFrame()
    
    def place_order(self, symbol, side, amount):
        """
        Place an order on Binance Futures testnet
        
        Parameters:
        -----------
        symbol : str
            Trading pair symbol (e.g., 'BTC/USDT')
        side : str
            'buy' or 'sell'
        amount : float
            Amount to trade
        
        Returns:
        --------
        dict
            Order response
        """
        try:
            # Set leverage for the symbol
            formatted_symbol = symbol.replace('/', '')
            try:
                self.exchange.fapiPrivatePostLeverage({
                    'symbol': formatted_symbol,
                    'leverage': self.leverage
                })
                print(f"Leverage for {symbol} set to {self.leverage}x")
            except Exception as e:
                print(f"Warning: Could not set leverage: {str(e)}")
                # Try alternative method for setting leverage
                try:
                    self.exchange.set_leverage(self.leverage, symbol)
                    print(f"Leverage for {symbol} set to {self.leverage}x (using alternative method)")
                except Exception as e2:
                    print(f"Warning: Could not set leverage using alternative method: {str(e2)}")
                print("Continuing with order placement...")
            
            # Place the order
            order = self.exchange.create_market_order(
                symbol=symbol,
                side=side.lower(),
                amount=amount
            )
            
            print(f"Placed {side.upper()} order for {amount} {symbol} successfully")
            return order
        except Exception as e:
            print(f"Error placing order: {str(e)}")
            return None
    
    def close_position(self, symbol):
        """
        Close an open position
        
        Parameters:
        -----------
        symbol : str
            Trading pair symbol (e.g., 'BTC/USDT')
        
        Returns:
        --------
        dict
            Order response
        """
        try:
            # Get position information
            positions = self.exchange.fetch_positions([symbol])
            if not positions or abs(float(positions[0]['contracts'])) == 0:
                print(f"No open position for {symbol}")
                return None
            
            # Determine side and amount for closing
            position_amount = float(positions[0]['contracts'])
            position_side = positions[0]['side']
            close_side = 'sell' if position_side == 'long' else 'buy'
            close_amount = abs(position_amount)
            
            # Place order to close position
            order = self.exchange.create_market_order(
                symbol=symbol,
                side=close_side,
                amount=close_amount,
                params={'reduceOnly': True}
            )
            
            print(f"Closed position for {symbol} successfully")
            return order
        except Exception as e:
            print(f"Error closing position: {str(e)}")
            return None
            
    def execute_strategy(self, days=20):
        """
        Execute the trading strategy based on identified signals
        
        Parameters:
        -----------
        days : int
            Number of days to calculate highs and lows (default: 20)
            
        Returns:
        --------
        tuple
            (executed_buys, executed_sells) as lists of order responses
        """
        # Get account balance
        balance = self.get_account_balance()
        print(f"Account balance: {balance} USDT")
        
        if balance <= 0:
            print("Insufficient account balance")
            return [], []
        
        # Identify signals
        buy_signals, sell_signals = self.identify_trading_signals(days=days)
        
        # Get current positions
        current_positions = self.get_open_positions()
        current_symbols = set(current_positions['symbol'].tolist()) if not current_positions.empty else set()
        
        executed_buys = []
        executed_sells = []
        
        # Execute sell signals (close long positions at 20-day lows)
        if not sell_signals.empty:
            for _, signal in sell_signals.iterrows():
                symbol = signal['market']
                if symbol in current_symbols:
                    print(f"Closing long position for {symbol} (20-day low)")
                    order = self.close_position(symbol)
                    if order:
                        executed_sells.append(order)
        
        # Execute buy signals (open long positions at 20-day highs)
        if not buy_signals.empty:
            for _, signal in buy_signals.iterrows():
                symbol = signal['market']
                current_price = signal['current_price']
                
                # Skip if already in position
                if symbol in current_symbols:
                    print(f"Already in position for {symbol}, skipping buy signal")
                    continue
                
                # Calculate position size
                position_size_usd = balance * self.position_size_pct
                amount = position_size_usd / current_price
                
                # Round to appropriate precision
                try:
                    market_info = self.exchange.market(symbol)
                    amount_precision = market_info['precision']['amount']
                    amount = round(amount, amount_precision)
                except:
                    # Default precision if market info not available
                    amount = round(amount, 4)
                
                # Place buy order
                print(f"Opening long position for {symbol} at {current_price} (20-day high)")
                order = self.place_order(symbol, 'buy', amount)
                if order:
                    executed_buys.append(order)
        
        return executed_buys, executed_sells
    
    def run_interactive(self):
        """Run interactive trading session"""
        print("==== Interactive Crypto Trading Strategy ====")
        print("Using Binance Futures Testnet")
        
        while True:
            print("\nMenu:")
            print("1. Get top cryptocurrencies by market cap")
            print("2. Fetch historical data")
            print("3. Show trading signals")
            print("4. Show account balance")
            print("5. Show open positions")
            print("6. Set leverage")
            print("7. Set position size")
            print("8. Execute strategy")
            print("9. Close specific position")
            print("10. Close all positions")
            print("0. Exit")
            
            choice = input("\nEnter your choice: ")
            
            try:
                if choice == '1':
                    limit = int(input("Enter number of top cryptocurrencies to fetch (default 50): ") or 50)
                    self.get_top_cryptos_by_marketcap(limit=limit)
                    if not self.top_cryptos_df.empty:
                        print("\nTop 10 cryptocurrencies by market cap:")
                        print(self.top_cryptos_df.head(10)[['symbol', 'name', 'market_cap', 'price']])
                
                elif choice == '2':
                    self.fetch_historical_data()
                    if self.historical_data:
                        print(f"\nCollected historical data for {len(self.historical_data)} cryptocurrencies")
                        print("\nAvailable symbols for trading:")
                        print(self.available_symbols)
                
                elif choice == '3':
                    days = int(input("Enter number of days for high/low calculation (default 20): ") or 20)
                    buy_signals, sell_signals = self.identify_trading_signals(days=days)
                    
                    print("\nBuy signals (cryptocurrencies at 20-day highs):")
                    if not buy_signals.empty:
                        print(buy_signals[['symbol', 'market', 'current_price', 'period_high']])
                    else:
                        print("No buy signals")
                        
                    print("\nSell signals (cryptocurrencies at 20-day lows):")
                    if not sell_signals.empty:
                        print(sell_signals[['symbol', 'market', 'current_price', 'period_low']])
                    else:
                        print("No sell signals")
                
                elif choice == '4':
                    balance = self.get_account_balance()
                    print(f"\nAccount balance: {balance} USDT")
                
                elif choice == '5':
                    positions = self.get_open_positions()
                    print("\nOpen positions:")
                    if not positions.empty:
                        print(positions)
                    else:
                        print("No open positions")
                
                elif choice == '6':
                    leverage = int(input("Enter leverage (1-20): "))
                    self.set_leverage(leverage)
                
                elif choice == '7':
                    pct = float(input("Enter position size as percentage of account balance (0-100): "))
                    self.set_position_size(pct)
                
                elif choice == '8':
                    days = int(input("Enter number of days for high/low calculation (default 20): ") or 20)
                    executed_buys, executed_sells = self.execute_strategy(days=days)
                    
                    print(f"\nExecuted {len(executed_buys)} buy orders and {len(executed_sells)} sell orders")
                
                elif choice == '9':
                    # Show available positions
                    positions = self.get_open_positions()
                    if positions.empty:
                        print("No open positions to close")
                        continue
                        
                    print("\nOpen positions:")
                    print(positions)
                    
                    symbol = input("Enter symbol to close (e.g., BTC/USDT): ")
                    self.close_position(symbol)
                
                elif choice == '10':
                    positions = self.get_open_positions()
                    if positions.empty:
                        print("No open positions to close")
                        continue
                        
                    for _, pos in positions.iterrows():
                        symbol = pos['symbol']
                        print(f"Closing position for {symbol}")
                        self.close_position(symbol)
                
                elif choice == '0':
                    print("Exiting...")
                    break
                
                else:
                    print("Invalid choice, please try again")
                    
            except Exception as e:
                print(f"Error: {str(e)}")
            
            # Pause before showing menu again
            input("\nPress Enter to continue...")

# Example usage
if __name__ == "__main__":
    # Get API keys from environment variables
    cmc_api_key = os.getenv('CMC_API_KEY')
    binance_api_key = os.getenv('BINANCE_FUTURES_TESTNET_API_KEY')
    binance_secret_key = os.getenv('BINANCE_FUTURES_TESTNET_SECRET_KEY')
    
    # Prompt for API keys if not found in environment
    if not cmc_api_key:
        cmc_api_key = input("Enter your CoinMarketCap API key: ")
    if not binance_api_key:
        binance_api_key = input("Enter your Binance Futures testnet API key: ")
    if not binance_secret_key:
        binance_secret_key = input("Enter your Binance Futures testnet secret key: ")
    
    # Initialize strategy
    strategy = CryptoTradingStrategy(
        cmc_api_key=cmc_api_key,
        binance_api_key=binance_api_key,
        binance_secret_key=binance_secret_key
    )
    
    # Run interactive session
    strategy.run_interactive()

Enter your CoinMarketCap API key:  1e15d5df-1e9a-4e36-883b-99491da567be
Enter your Binance Futures testnet API key:  cde18225162872ffca75eaa9c4fce4090345009928497c9966bf875a06fb4218
Enter your Binance Futures testnet secret key:  8780d93f731bed0108b4413a387acada57d96736baabe08ead440241ba077fd8


==== Interactive Crypto Trading Strategy ====
Using Binance Futures Testnet

Menu:
1. Get top cryptocurrencies by market cap
2. Fetch historical data
3. Show trading signals
4. Show account balance
5. Show open positions
6. Set leverage
7. Set position size
8. Execute strategy
9. Close specific position
10. Close all positions
0. Exit



Enter your choice:  1
Enter number of top cryptocurrencies to fetch (default 50):  


Retrieved top 50 cryptocurrencies by market cap

Top 10 cryptocurrencies by market cap:
  symbol         name    market_cap         price
0    BTC      Bitcoin  1.671612e+12  84208.112149
1    ETH     Ethereum  1.938431e+11   1606.149328
2   USDT  Tether USDt  1.442923e+11      0.999645
3    XRP          XRP  1.240852e+11      2.126999
4    BNB          BNB  8.329549e+10    584.650629
5    SOL       Solana  6.684316e+10    129.517982
6   USDC         USDC  6.004789e+10      0.999787
7   DOGE     Dogecoin  2.442578e+10      0.164106
8    TRX         TRON  2.419907e+10      0.254829
9    ADA      Cardano  2.265459e+10      0.642118



Press Enter to continue... 


...


Open positions:
          symbol  side   entry_price  amount         pnl    roe
0  BTC/USDT:USDT  LONG  78330.652333     0.3  1881.57558  74.13



Press Enter to continue... 



Menu:
1. Get top cryptocurrencies by market cap
2. Fetch historical data
3. Show trading signals
4. Show account balance
5. Show open positions
6. Set leverage
7. Set position size
8. Execute strategy
9. Close specific position
10. Close all positions
0. Exit



Enter your choice:  0


Exiting...


In [ ]:
1e15d5df-1e9a-4e36-883b-99491da567be
cde18225162872ffca75eaa9c4fce4090345009928497c9966bf875a06fb4218
8780d93f731bed0108b4413a387acada57d96736baabe08ead440241ba077fd8
